In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

# Toy example 1 - Single eligible group, Single ineligible group

In [2]:
def _extract_solution_discount(m):
    """
    Get solution from optimization model
    """
    users_in = [v.x for v in m.getVars() if v.VarName.find("ineligible_val") != -1]
    users_el = [v.x for v in m.getVars() if v.VarName.find("eligible_allocation") != -1]
    edge_flows = [v.x for v in m.getVars() if v.VarName.find("edge_flows") != -1]
    
    users_in = np.reshape(users_in, (2, T))
    users_el = np.reshape(users_el, (2, T))
    edge_flows = np.reshape(edge_flows, (2, T))

    solution = {
        "users_in": users_in,
        "users_el": users_el,
        "edge_flows": edge_flows
    }
    
    return solution

In [3]:
def avg_travel_times_1(solution_set, theta):
    
    users_in = solution_set["users_in"]
    users_el = solution_set["users_el"]
    edge_flows = solution_set["edge_flows"]
    
    avg_travel_time_in = sum( users_in[j, t] * (theta[j,0] * edge_flows[j, t] + theta[j,1]) for j in range(2) for t in range(T)) / T
    avg_travel_time_el = sum( users_el[j, t] * (theta[j,0] * edge_flows[j, t] + theta[j,1]) for j in range(2) for t in range(T)) / T
    avg_travel_time_all = sum( edge_flows[j, t] * (theta[j,0] * edge_flows[j, t] + theta[j,1]) for j in range(2) for t in range(T)) / (T*2)
    
    avg_travel_times = {
        "users_in": avg_travel_time_in,
        "users_el": avg_travel_time_el,
        "all": avg_travel_time_all
    }
    
    return avg_travel_times


In [4]:
def m_d1_solve(theta, T, toll, vot_ineligible, vot_eligible, discount_ratio):
    a_1 = theta[0, 0]
    b_1 = theta[0, 1]
    a_2 = theta[1, 0]
    b_2 = theta[1, 1]
    
    # Initialize model
    m_d1 = gp.Model()

    users_in = m_d1.addVars(2, T, name = 'ineligible_val', lb = -10000.0, ub = 10000.0)
    edge_flows = m_d1.addVars(2, T, name = 'edge_flows', lb = -10000.0, ub = 10000.0)
    users_el = m_d1.addVars(2, T, name = 'eligible_allocation', lb = -10000.0, ub = 10000.0)

    ## Add constraints to Model 

    # Total edge flow, express and general purpose lanes:
    m_d1.addConstrs((edge_flows[j, t] == users_in[j, t] + users_el[j, t] for j in range(2) for t in range(T)))

    # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
    m_d1.addConstrs((users_in[j, t]>=0 for j in range(2) for t in range(T)))
    m_d1.addConstrs((users_el[j, t]>=0 for j in range(2) for t in range(T)))

    # Every eligible and ineligible user is assigned to one of three options: \
    # (express paying, express with credit, non-express)
    m_d1.addConstrs((sum(users_in[j, t] for j in range(2)) == 1 for t in range(T)))
    m_d1.addConstrs((sum(users_el[j, t] for j in range(2)) == 1 for t in range(T)))

    ## To edit below:
    # Set Objective
    m_d1.setObjective(sum( a_1*(edge_flows[0, t]) + b_1*(edge_flows[0, t]**2)/2  
                        + a_2*(edge_flows[1, t]) + b_2*(edge_flows[1, t]**2)/2 
                        + toll*users_in[0,t]/vot_ineligible
                        + (1 - discount_ratio)*toll*users_el[0,t]/vot_eligible 
        for t in range(T)), GRB.MINIMIZE)
    
    m_d1.update()

    m_d1.optimize()
    
    solution_set_d1 = _extract_solution_discount(m_d1)

    return solution_set_d1

In [5]:
def m_b1_solve(theta, T, toll, vot_ineligible, vot_eligible, budget):
    a_1 = theta[0, 0]
    b_1 = theta[0, 1]
    a_2 = theta[1, 0]
    b_2 = theta[1, 1]
    
    # Initialize model
    m_b1 = gp.Model()

    users_in = m_b1.addVars(2, T, name = 'ineligible_val', lb = -10000.0, ub = 10000.0)
    edge_flows = m_b1.addVars(2, T, name = 'edge_flows', lb = -10000.0, ub = 10000.0)
    users_el = m_b1.addVars(2, T, name = 'eligible_allocation', lb = -10000.0, ub = 10000.0)

    ## Add constraints to Model 

    # Total edge flow, express and general purpose lanes:
    m_b1.addConstrs((edge_flows[j, t] == users_in[j, t] + users_el[j, t] for j in range(2) for t in range(T)))

    # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
    m_b1.addConstrs((users_in[j, t]>=0 for j in range(2) for t in range(T)))
    m_b1.addConstrs((users_el[j, t]>=0 for j in range(2) for t in range(T)))
    
    # Budget constraint satisfaction:
    if toll != 0:
        m_b1.addConstr((sum(toll*users_el[0, t] for t in range(T)) <= budget))

    # Every eligible and ineligible user is assigned to one of three options: \
    # (express paying, express with credit, non-express)
    m_b1.addConstrs((sum(users_in[j, t] for j in range(2)) == 1 for t in range(T)))
    m_b1.addConstrs((sum(users_el[j, t] for j in range(2)) == 1 for t in range(T)))

    ## To edit below:
    # Set Objective
    m_b1.setObjective(sum( a_1*(edge_flows[0, t]) + b_1*(edge_flows[0, t]**2)/2  
                        + a_2*(edge_flows[1, t]) + b_2*(edge_flows[1, t]**2)/2 
                        + toll*users_in[0,t]/vot_ineligible
        for t in range(T)), GRB.MINIMIZE)
    
    m_b1.update()

    m_b1.optimize()
    
    solution_set_b1 = _extract_solution_discount(m_b1)

    return solution_set_b1

In [30]:
time_1 = time.time()

# Affine lane functions:
a_1 = 1.0
b_1 = 1.0
a_2 = 1.0
b_2 = 1.0
theta = np.array([[a_1, b_1], [a_2, b_2]])
T = 1
# toll = 1.0
# budget = 0.25
toll_array = (np.arange(25) + 1)/25
budget_array = (np.arange(25) + 1)*T/25

vot_ineligible = 1
vot_eligible = 1

solution_set_array_d1 = {}
solution_set_array_b1 = {}

# Return solution:

for toll_idx, toll in enumerate(toll_array):
    
    solution_set_array_at_toll_d1 = {}
    solution_set_array_at_toll_b1 = {}
    
    for budget_idx, budget in enumerate(budget_array):
        
        if budget > toll * T:
            continue
        
        discount_ratio = budget / (toll * T)
        
        print()
        print("Toll:", toll)
        print("Budget:", budget)
        print("Discount Ratio:", discount_ratio)
        print()
        
        solution_set_d1 = m_d1_solve(theta, T, toll, vot_ineligible, vot_eligible, discount_ratio)
        solution_set_b1 = m_b1_solve(theta, T, toll, vot_ineligible, vot_eligible, budget)
        
        solution_set_array_at_toll_d1[budget] = solution_set_d1
        solution_set_array_at_toll_b1[budget] = solution_set_b1
    
    solution_set_array_d1[toll] = solution_set_array_at_toll_d1
    solution_set_array_b1[toll] = solution_set_array_at_toll_b1

time_2 = time.time()
print("Time: ", time_2 - time_1)


Toll: 0.04
Budget: 0.04
Discount Ratio: 1.0

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 6 columns and 14 nonzeros
Model fingerprint: 0x910da088
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-02, 1e+00]
  QObjective range [1e+00, 1e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual


                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.23465801e+07 -1.16884646e+08  1.42e+04 3.73e+03  6.05e+06     0s
   1   3.74621342e+01 -2.16747959e+06  1.27e+01 3.33e+00  2.65e+05     0s
   2   3.04001473e+00 -3.70068894e+03  1.27e-05 3.33e-06  4.63e+02     0s
   3   3.03999214e+00 -8.47786395e-01  6.31e-10 1.66e-10  4.86e-01     0s
   4   3.03723608e+00  2.96990110e+00  7.28e-12 9.99e-16  8.42e-03     0s
   5   3.00030260e+00  2.99591929e+00  1.82e-12 1.82e-12  5.48e-04     0s
   6   3.00005405e+00  2.99992818e+00  1.82e-12 1.12e-16  1.57e-05     0s
   7   3.00000559e+00  2.99999379e+00  0.00e+00 0.00e+00  1.47e-06     0s
   8   3.00000079e+00  2.99999912e+00  0.00e+00 1.82e-12  2.12e-07     0s
   9   3.00000014e+00  2.99999987e+00  1.82e-12 5.26e-17  2.96e-08     0s
  10   3.00000003e+00  2.99999999e+00  3.64e-12 1.78e-12  4.18e-09     0s

Barrier solved model in 10 iterations and 0.04 seconds (0.


CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 6 columns and 15 nonzeros
Model fingerprint: 0xeea9fec3
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e-01, 1e+00]
  QObjective range [1e+00, 1e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [8e-02, 1e+00]
Presolve removed 7 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.12147071e+07 -1.13327769e+08  4.60e+03 3.60e+03  5.96e+06     0s
   1   3.87062017e+01 -2.09719324e+0

Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.01952808e+07 -1.10068184e+08  4.49e+03 3.49e+03  5.88e+06     0s
   1   4.08335546e+01 -2.03371215e+06  4.54e+00 3.51e+00  2.48e+05     0s
   2   3.20908097e+00 -3.01038879e+03  4.52e-06 3.51e-06  3.77e+02     0s
   3   3.20868409e+00 -8.93203598e-01  1.63e-09 1.27e-09  5.13e-01     0s
   4   3.13478065e+00  2.99648494e+00  2.55e-11 1.82e-11  1.73e-02     0s
   5   3.11733750e+00  3.11181495e+00  1.82e-12 1.82e-12  6.90e-04     0s
   6   3.11360374e+00  3.11359176e+00  0.00e+00 5.55e-17  1.50e-06     0s
   7   3.11359999e+00  3.11359999e+00  1.82e-12 0.00e+00  1.50e-09     0s

Barrier solved model in 7 iterations and 0.02 seconds (0.00 work units)
Optimal objective 3.11359999e+00


Toll: 0.16
Budget: 0.08
Discount Ratio: 0.5

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical p

  Objective range  [2e-01, 1e+00]
  QObjective range [1e+00, 1e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.01953177e+07 -1.10069053e+08  1.35e+04 3.49e+03  5.88e+06     0s
   1   3.43515335e+01 -2.02420151e+06  1.20e+01 3.10e+00  2.47e+05     0s
   2   3.08000780e+00 -3.54225541e+03  1.20e-05 3.10e-06  4.43e+02     0s
   3   3.07996831e+00 -8.22059017e-01  1.21e-09 3.13e-10  4.88e-01     0s
   4   3.06936965e+00  2.95171693e+00  2.36e-11 7.28e-12  1.47e-02     0s
   5   3.00100419e+00  2.99702049e+00  1.82e-12 0.00e+

...

  RHS range        [3e-01, 1e+00]
Presolve removed 7 rows and 2 columns
Presolve time: 0.03s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.04793630e+07 -7.52311284e+07  3.24e+03 2.24e+03  4.93e+06     0s
   1   2.34958583e+01 -1.27609931e+06  3.20e+00 2.21e+00  1.56e+05     0s
   2   3.61745211e+00 -2.26895804e+03  3.20e-06 2.21e-06  2.84e+02     0s
   3   3.61720274e+00  5.84474713e-01  1.08e-09 7.42e-10  3.79e-01     0s
   4   3.49539310e+00  3.29757389e+00  4.37e-11 2.91e-11  2.47e-02     0s
   5   3.43071571e+00  3.41570574e+00  3.64e-12 1.82e-12  1.88e-03     0s
   6   3.43000264e+00  3.42976949e+00  1.82e-12 1.82e-12  2.91e-05     0s
   7   3


Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.04794353e+07 -7.52317843e+07  3.24e+03 2.24e+03  4.93e+06     0s
   1   2.11635872e+01 -1.27160246e+06  2.96e+00 2.04e+00  1.55e+05     0s
   2   3.52956514e+00 -2.50830857e+03  2.96e-06 2.04e-06  3.14e+02     0s
   3   3.52898905e+00 -2.13073254e-01  1.45e-09 1.00e-09  4.68e-01     0s
   4   3.30250660e+00  2.95225109e+00  9.09e-11 6.18e-11  4.38e-02     0s
   5   3.11485156e+00  3.08830778e+00  5.46e-12 3.64e-12  3.32e-03     0s
   6   3.10273845e+00  3.10237381e+00  0.00e+00 2.22e-16  4.56e-05     0s
   7   3.10240033e+00  3.10239999e+00  0.00e+00 1.09e-12  4.76e-08     0s
   8   3.10240000e+00  3.10239999e+00  0.00e+00 1.82e-12  4.77e-11     0s

Barrier solved model in 8 iterations and 0.02 seconds (0.00 wo

Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 6 columns and 14 nonzeros
Model fingerprint: 0xd86bf203
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+00]
  QObjective range [1e+00, 1e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 2 columns
Presolve time: 0.11s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.04788571e+07 -7.52323860e+07  9.71e+03 2.24e+03  4.93e+06     0s
   1   1.87492008e+01 -1.26760608e+06  8.21e+00 1.89e+00  1.55e+05     0s
   2   3.599689

   5   3.12118899e+00  3.08836503e+00  1.46e-11 3.64e-12  4.10e-03     0s
   6   3.11662338e+00  3.11445067e+00  0.00e+00 0.00e+00  2.72e-04     0s
   7   3.11640115e+00  3.11632026e+00  0.00e+00 0.00e+00  1.01e-05     0s
   8   3.11640001e+00  3.11639992e+00  0.00e+00 1.82e-12  1.00e-08     0s
   9   3.11640000e+00  3.11640001e+00  0.00e+00 2.22e-16  1.00e-11     0s

Barrier solved model in 9 iterations and 0.02 seconds (0.00 work units)
Optimal objective 3.11640000e+00

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 6 columns and 15 nonzeros
Model fingerprint: 0x61da2c11
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [1e+00, 1e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [9e-

  RHS range        [1e+00, 1e+00]
Presolve removed 7 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 6 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.04794953e+07 -7.52323099e+07  3.24e+03 2.24e+03  4.93e+06     0s
   1   1.94365910e+01 -1.26810500e+06  2.76e+00 1.91e+00  1.55e+05     0s
   2   3.50117821e+00 -2.64302254e+03  2.76e-06 1.91e-06  3.31e+02     0s
   3   3.50020149e+00 -9.14621800e-01  1.85e-09 1.28e-09  5.52e-01     0s
   4   3.22643375e+00  2.85604599e+00  8.73e-11 6.00e-11  4.63e-02     0s
   5   3.01351067e+00  2.98217270e+00  5.46e-12 3.64e-12  3.92e-03     0s
   6   3.00354066e+00  3.00110886e+00  1.82e-12 1.11e-16  3.04e-04     0s
   7   3

In [34]:
# Average travel times, discount minus budget:

avg_travel_time_discount_minus_budget = {}
avg_travel_time_discount_minus_budget_values = []

for toll_idx, toll in enumerate(toll_array):
    
    avg_travel_time_discount_minus_budget_at_toll = {}
    
    for budget_idx, budget in enumerate(budget_array):        
        if budget > toll * T:
            continue
            
#         print("toll:", toll)
#         print("budget:", budget)
#         print()
            
        diff = avg_travel_times_1(solution_set_array_d1[toll][budget], theta)["users_el"] - avg_travel_times_1(solution_set_array_b1[toll][budget], theta)["users_el"]
        avg_travel_time_discount_minus_budget_at_toll[budget] = diff
        avg_travel_time_discount_minus_budget_values.append(diff)
    
    avg_travel_time_discount_minus_budget[toll] = avg_travel_time_discount_minus_budget_at_toll    

avg_travel_time_discount_minus_budget

{0.04: {0.04: 0.0},
 0.08: {0.04: -0.019200389289346642, 0.08: -1.8185453143360064e-12},
 0.12: {0.04: -0.056800052672278056, 0.08: 0.0007999781840737352, 0.12: 0.0},
 0.16: {0.04: -0.09280001160539131,
  0.08: -0.03680000386866267,
  0.12: 0.020799886206726903,
  0.16: 0.0},
 0.2: {0.04: -0.1272000056697613,
  0.08: -0.07280009637227525,
  0.12: -0.016800003407506203,
  0.16: 0.04079967393884787,
  0.2: 0.0},
 0.24: {0.04: -0.1600000060734481,
  0.08: -0.1072000255232719,
  0.12: -0.05280000038444044,
  0.16: 0.003199964012361445,
  0.2: 0.0607988603068117,
  0.24: 0.0},
 0.28: {0.04: -0.19120000587249608,
  0.08: -0.14000000436209215,
  0.12: -0.08720010096781294,
  0.16: -0.032800002756792734,
  0.2: 0.02319996260564361,
  0.24: 0.08079680593951388,
  0.28: -8.494921048818327e-05},
 0.32: {0.04: -0.22080000530680222,
  0.08: -0.17120000446046402,
  0.12: -0.12000002384350594,
  0.16: -0.06720000035487272,
  0.2: -0.012800002028198199,
  0.24: 0.04319990956545361,
  0.28: 0.074549706

In [35]:
print("Max average time difference, discount minus budget, value:", max(avg_travel_time_discount_minus_budget_values))
print("Min average time difference, discount minus budget, value:", min(avg_travel_time_discount_minus_budget_values))

Max average time difference, discount minus budget, value: 0.08315280249715151
Min average time difference, discount minus budget, value: -0.47919999918503375


In [9]:
edge_flows_d1 = solution_set_d1["edge_flows"]
users_el_d1 = solution_set_d1["users_el"]
users_in_d1 = solution_set_d1["users_in"]

edge_flows_b1 = solution_set_b1["edge_flows"]
users_el_b1 = solution_set_b1["users_el"]
users_in_b1 = solution_set_b1["users_in"]

print("edge_flows_d1:\n", edge_flows_d1)
print()
print("users_el_d1:\n", users_el_d1)
print()
print("users_in_d1:\n", users_in_d1)
print()

print()
print("edge_flows_b1:\n", edge_flows_b1)
print()
print("users_el_b1:\n", users_el_b1)
print()
print("users_in_b1:\n", users_in_b1)
print()

avg_travel_times_d1 = avg_travel_times_1(solution_set_d1, theta)
avg_travel_times_b1 = avg_travel_times_1(solution_set_b1, theta)

print()
print("avg_travel_times_d1:\n", avg_travel_times_d1)
print()
print("avg_travel_times_b1:\n", avg_travel_times_b1)
print()

edge_flows_d1:
 [[0.99986977]
 [1.00013023]]

users_el_d1:
 [[9.99869767e-01]
 [1.30232747e-04]]

users_in_d1:
 [[8.33335623e-11]
 [1.00000000e+00]]


edge_flows_b1:
 [[0.99986977]
 [1.00013023]]

users_el_b1:
 [[9.99869767e-01]
 [1.30232747e-04]]

users_in_b1:
 [[8.33335623e-11]
 [1.00000000e+00]]


avg_travel_times_d1:
 {'users_in': 2.000130232663629, 'users_el': 1.9998698012574647, 'all': 2.0000000169605467}

avg_travel_times_b1:
 {'users_in': 2.000130232663629, 'users_el': 1.9998698012574643, 'all': 2.0000000169605467}



# From other files:

In [ ]:
# # Solver 2, Discount:

# def OptPL_discount(toll, discount_ratio, T = T, num_eligible = num_eligible, num_ineligible = num_ineligible, 
#             vot_ineligible = vot_ineligible, alpha = bpr_true_alpha, fftt_1 = bpr_true_fftt, 
#               fftt_2 = bpr_true_fftt, c_1 = bpr_true_capacity, c_2 = bpr_true_capacity, 
#              a = const_multiplier, cap_thresh = cap_thresh_multiplier, b_1 = apx_slope, b_2 = apx_slope):
#     """
#     Function to solve convex optimization problem given a particular toll and budget value
#     """
    
#     # Initialize model
#     m1 = gp.Model()
#     # Add variables to model
# #     users_in = m1.addVars(num_ineligible, 3, T, name = 'ineligible_val', lb = 0.0, ub = 2.0)
# #     edge_flows = m1.addVars(2, T, name = 'edge_flows', lb = 0.0, ub = num_ineligible * 2.0)
# #     users_el = m1.addVars(3, T, name = 'eligible_allocation', lb = 0.0, ub = 2.0)
#     # Add variables for piecewise affine approximation
# #     eps_flows = m1.addVars(2, T, name = 'eps_flows', lb = 0.0, ub = num_ineligible * 2.0)
    
# #     # Add variables for piecewise affine approximation
# #     eps_flows = m1.addVars(2, T, name = 'eps_flows', lb = 0.0, ub = num_ineligible * 2.0)
#     users_in = m1.addVars(num_ineligible, 2, T, name = 'ineligible_val', lb = -10000.0, ub = 10000.0)
#     edge_flows = m1.addVars(2, T, name = 'edge_flows', lb = -10000.0, ub = 10000.0)
#     users_el = m1.addVars(2, T, name = 'eligible_allocation', lb = -10000.0, ub = 10000.0)
#     eps_flows = m1.addVars(2, T, name = 'eps_flows', lb = -10000.0, ub = 10000.0)   

#     ## Add constraints to Model 
    
#     # Total edge flow, express and general purpose lanes:
#     m1.addConstrs((edge_flows[j, t] == sum(users_in[i, j, t] for i in range(num_ineligible)) + \
#                   users_el[j, t] for j in range(2) for t in range(T)))
    
#     # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
#     m1.addConstrs((users_in[i, j, t]>=0 for i in range(num_ineligible) for j in range(2) for t in range(T)))
#     m1.addConstrs((users_el[j, t]>=0 for j in range(2) for t in range(T)))
    
#     # Every eligible and ineligible user is assigned to one of three options: \
#     # (express paying, express with credit, non-express)
#     m1.addConstrs((sum(users_in[i, j, t] for j in range(2)) == 1 for i in range(num_ineligible) for t in range(T)))
#     m1.addConstrs((sum(users_el[j, t] for j in range(2)) == num_eligible for t in range(T)))
        
#     # Piecewise affine approximation:
#     m1.addConstrs((eps_flows[j, t] >= 0 for j in range(2) for t in range(T))) # Must be at least 0
#     m1.addConstrs((eps_flows[0, t] >= edge_flows[0, t] - cap_thresh*c_1 for t in range(T))) # Must be at least flow - capacity
#     m1.addConstrs((eps_flows[1, t] >= edge_flows[1, t] - cap_thresh*c_2 for t in range(T))) # Must be at least flow - capacity
    
    
#     ## To edit below:
#     # Set Objective
#     m1.setObjective(sum( a*fftt_1*(edge_flows[0, t]) + b_1*(eps_flows[0, t]**2)/2  
#                         + a*fftt_2*(edge_flows[1, t]) + b_2*(eps_flows[1, t]**2)/2 
#                         + sum( toll*users_in[i,0,t]/vot_ineligible[i,t] for i in range(num_ineligible))
#                         + sum( (1 - discount_ratio)*toll*users_el[0,t]/vot_eligible[i,t] for i in range(num_eligible))
#         for t in range(T)), GRB.MINIMIZE)
    
#     m1.update()
    
#     return m1

# Scratch Work:

In [ ]:
# m_sample = gp.Model()
